In [11]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

# Load the API key from .env file
load_dotenv('/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/API Keys/.env')
api_key = os.getenv('Census_API_Key')

# List of Ohio counties (to get all census tracts in Ohio)
ohio_counties = [
    "001", "003", "005", "007", "009", "011", "013", "015", "017", "019",
    "021", "023", "025", "027", "029", "031", "033", "035", "037", "039",
    "041", "043", "045", "047", "049", "051", "053", "055", "057", "059",
    "061", "063", "065", "067", "069", "071", "073", "075", "077", "079",
    "081", "083", "085", "087", "089", "091", "093", "095", "097", "099",
    "101", "103", "105", "107", "109", "111", "113", "115", "117", "119",
    "121", "123", "125", "127", "129", "131", "133", "135", "137", "139",
    "141", "143", "145", "147", "149", "151", "153", "155", "157", "159",
    "161", "163", "165", "167", "169", "171", "173", "175"
]

# Initialize an empty DataFrame to hold all data
df_all = pd.DataFrame()

# Construct the base URL for ACS 5-Year 2020 data for Educational Attainment (S1501)
base_url = (
    'https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,'
    'S1501_C01_001E,S1501_C01_002E,S1501_C01_003E,S1501_C01_004E,'
    'S1501_C01_005E,S1501_C01_006E,S1501_C01_007E,S1501_C01_008E,'
    'S1501_C01_009E,S1501_C01_010E,S1501_C01_011E,S1501_C01_012E,'
    'S1501_C01_013E,S1501_C01_014E,S1501_C01_015E,S1501_C01_016E,'
    'S1501_C01_017E,S1501_C01_018E,S1501_C01_019E,S1501_C01_020E,'
    'S1501_C01_021E,S1501_C01_022E'
)

# Initialize columns variable
columns = []

# Loop through counties and make API requests for each census tract in the county
for county in ohio_counties:
    # Construct the URL for each county
    url = f"{base_url}&for=tract:*&in=state:39%20county:{county}&key={api_key}"
    
    # Make the API request
    response = requests.get(url)
    
    # Check the status code of the response
    if response.status_code == 200:
        try:
            data = response.json()
            
            # Check if data is empty
            if len(data) > 1:
                # Get columns from the first valid response
                if not columns:
                    columns = data[0]
                
                # Create a DataFrame from the current county's data
                df_current = pd.DataFrame(data[1:], columns=columns)
                
                # Append the current county's data to the main DataFrame
                df_all = pd.concat([df_all, df_current], ignore_index=True)
            else:
                print(f"No data for county: {county}")
        
        except ValueError as e:
            print(f"Error decoding JSON for county: {county} {e}")
            print("Response content:", response.content)
    else:
        print(f"Request failed for county: {county} with status code: {response.status_code}")
        print("Response content:", response.content)

# Convert numeric columns to numbers
if not df_all.empty:
    for column in columns[1:-1]:  # Exclude 'NAME' and 'tract'
        df_all[column] = pd.to_numeric(df_all[column])

# Display the DataFrame using Pandas
print(df_all)

# Export the DataFrame to a CSV file
output_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Educational_Attainment_Ohio_Census_Tract_2020.csv'
df_all.to_csv(output_path, index=False)


                                          NAME  S1501_C01_001E  \
0        Census Tract 7701, Adams County, Ohio             340   
1        Census Tract 7702, Adams County, Ohio             311   
2     Census Tract 7703.01, Adams County, Ohio             259   
3     Census Tract 7703.02, Adams County, Ohio             223   
4        Census Tract 7704, Adams County, Ohio             353   
...                                        ...             ...   
3163   Census Tract 9381, Wyandot County, Ohio             377   
3164   Census Tract 9382, Wyandot County, Ohio             235   
3165   Census Tract 9383, Wyandot County, Ohio             240   
3166   Census Tract 9384, Wyandot County, Ohio             371   
3167   Census Tract 9385, Wyandot County, Ohio             155   

      S1501_C01_002E  S1501_C01_003E  S1501_C01_004E  S1501_C01_005E  \
0                 40             213              84               3   
1                 24              85              92           